# MCM2017 B

## Problem B: Merge After Toll
Multi-lane divided limited-access toll highways use “ramp tolls” and “barrier tolls” to collect tolls from motorists. A ramp toll is a collection mechanism at an entrance or exit ramp to the highway and these do not concern us here. A barrier toll is a row of tollbooths placed across the highway, perpendicular to the direction of traffic flow. There are usually (always) more tollbooths than there are incoming lanes of traffic (see former 2005 MCM Problem B). So when exiting the tollbooths in a barrier toll, vehicles must “fan in” from the larger number of tollbooth egress lanes to the smaller number of regular travel lanes. A toll plaza is the area of the highway needed to facilitate the barrier toll, consisting of the fan-out area before the barrier toll, the toll barrier itself, and the fan-in area after the toll barrier. For example, a three-lane highway (one direction) may use 8 tollbooths in a barrier toll. After paying toll, the vehicles continue on their journey on a highway having the same number of lanes as had entered the toll plaza (three, in this example). 

Consider a toll highway having L lanes of travel in each direction and a barrier toll containing B tollbooths (B > L) in each direction. Determine the shape, size, and merging pattern of the area following the toll barrier in which vehicles fan in from B tollbooth egress lanes down to L lanes of traffic. Important considerations to incorporate in your model include accident prevention, throughput (number of vehicles per hour passing the point where the end of the plaza joins the L outgoing traffic lanes), and cost (land and road construction are expensive). In particular, this problem does not ask for merely a performance analysis of any particular toll plaza design that may already be implemented. The point is to determine if there are better solutions (shape, size, and merging pattern) than any in common use. 

Determine the performance of your solution in light and heavy traffic. How does your solution change as more autonomous (self-driving) vehicles are added to the traffic mix? How is your solution affected by the proportions of conventional (human-staffed) tollbooths, exact-change (automated) tollbooths, and electronic toll collection booths (such as electronic toll collection via a transponder in the vehicle)?


### Link 2005 MCM Problem B （Not Problem-B 2017）
PROBLEM B: Tollbooths

Heavily-traveled toll roads such as the Garden State Parkway , Interstate 95, and so forth, are multi-lane divided highways that are interrupted at intervals by toll plazas. Because collecting tolls is usually unpopular, it is desirable to minimize motorist annoyance by limiting the amount of traffic disruption caused by the toll plazas. Commonly, a much larger number of tollbooths is provided than the number of travel lanes entering the toll plaza. Upon entering the toll plaza, the flow of vehicles fans out to the larger number of tollbooths, and when leaving the toll plaza, the flow of vehicles is required to squeeze back down to a number of travel lanes equal to the number of travel lanes before the toll plaza. Consequently, when traffic is heavy, congestion increases upon departure from the toll plaza. When traffic is very heavy, congestion also builds at the entry to the toll plaza because of the time required for each vehicle to pay the toll.

Make a model to help you determine the optimal number of tollbooths to deploy in a barrier-toll plaza. Explicitly consider the scenario where there is exactly one tollbooth per incoming travel lane. Under what conditions is this more or less effective than the current practice? Note that the definition of “optimal” is up to you to determine.

## 探索
### 1. 数据1: [link](http://metrocosm.com/get-the-data/#interstatetraffic)

In [43]:
import pandas as pd

In [52]:
datas = pd.read_csv('http://metrocosm.com/wp-content/uploads/2015/03/vol2015.csv', skiprows=[0])

/home/nouzan/git/Modash/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
datas.shape

(580025, 36)

In [54]:
datas.head()

,key,year_record,month_record,day_record,day_of_week,state_code,f_system,station_id,travel_dir,travel_lane,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,restrictions
0,7-2-101,2015,11,14,7,2,1,101,1,1,...,22,13,13,14,7,5,6,1,5,NaN
1,7-2-101,2015,11,14,7,2,1,101,5,1,...,28,34,26,10,18,13,9,4,6,NaN
2,1-2-101,2015,11,15,1,2,1,101,1,1,...,22,28,20,19,18,9,7,6,6,NaN
3,1-2-101,2015,11,15,1,2,1,101,5,1,...,36,51,40,23,15,14,10,6,6,NaN
4,2-2-101,2015,11,16,2,2,1,101,1,1,...,17,15,11,14,13,9,10,15,13,NaN


In [55]:
datas.dtypes

key             object
year_record     object
month_record    object
day_record      object
day_of_week     object
state_code      object
f_system        object
station_id      object
travel_dir      object
travel_lane     object
data_source     object
hour_00         object
hour_01         object
hour_02         object
hour_03         object
hour_04         object
hour_05         object
hour_06         object
hour_07         object
hour_08         object
hour_09         object
hour_10         object
hour_11         object
hour_12         object
hour_13         object
hour_14         object
hour_15         object
hour_16         object
hour_17         object
hour_18         object
hour_19         object
hour_20         object
hour_21         object
hour_22         object
hour_23         object
restrictions    object
dtype: object

In [56]:
datas.describe()

,key,year_record,month_record,day_record,day_of_week,state_code,f_system,station_id,travel_dir,travel_lane,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,restrictions
count,580025,580025,580025,580025,580025,580025,580025,580025,580025,580025,...,580025,580025,580025,580025,580025,580025,580025,580025,580025,49
unique,42491,3,3,61,15,58,24,4841,20,19,...,13505,13714,13473,12017,10101,8670,7807,6668,5189,1
top,2-15-000C7L,2015,11,8,2,12,11,50,1,1,...,0,0,0,0,0,0,0,0,0,restrictions
freq,130,409551,409551,14095,67084,49152,88453,1307,116282,155595,...,3460,3489,3733,3894,4297,4798,5676,7288,11586,49


In [57]:
datas.day_of_week.value_counts()

2              67084
1              66966
7              55562
5              55183
4              55131
6              54836
3              54789
1              28431
2              28121
7              22903
4              22891
3              22739
5              22703
6              22637
day_of_week       49
Name: day_of_week, dtype: int64

In [65]:
datas.columns.str.endswith('_record')

array([False,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [58]:
columns = datas.columns

In [59]:
dtypes_dict = {}

In [63]:
for col in columns:
    if str(col).startswith('hour_'):
        dtypes_dict[str(col)] = 'int'

In [67]:
dtypes_dict['year_record'] = 'int'
dtypes_dict['month_record'] = 'int'
dtypes_dict['day_record'] = 'int'
dtypes_dict['day_of_week'] = 'int'
dtypes_dict['station_id'] = 'int'
dtypes_dict['travel_dir'] = 'int'
dtypes_dict['travel_lane'] = 'int'

In [70]:
datas = pd.read_csv('http://metrocosm.com/wp-content/uploads/2015/03/vol2015.csv', skiprows=[0])

/home/nouzan/git/Modash/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
datas[list(dtypes_dict.keys())] = datas[list(dtypes_dict.keys())].apply(pd.to_numeric, errors='coerce', downcast='integer')

In [84]:
datas.dtypes

key              object
year_record     float64
month_record    float64
day_record      float64
day_of_week     float64
state_code       object
f_system         object
station_id      float64
travel_dir      float64
travel_lane     float64
data_source      object
hour_00         float64
hour_01         float64
hour_02         float64
hour_03         float64
hour_04         float64
hour_05         float64
hour_06         float64
hour_07         float64
hour_08         float64
hour_09         float64
hour_10         float64
hour_11         float64
hour_12         float64
hour_13         float64
hour_14         float64
hour_15         float64
hour_16         float64
hour_17         float64
hour_18         float64
hour_19         float64
hour_20         float64
hour_21         float64
hour_22         float64
hour_23         float64
restrictions     object
dtype: object

In [88]:
datas.drop('key', axis=1, inplace=True)

In [91]:
datas.drop(['state_code', 'data_source', 'restrictions'], axis=1, inplace=True)

In [92]:
datas.head()

,year_record,month_record,day_record,day_of_week,f_system,station_id,travel_dir,travel_lane,hour_00,hour_01,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2015.0,11.0,14.0,7.0,1,101.0,1.0,1.0,6.0,1.0,...,23.0,22.0,13.0,13.0,14.0,7.0,5.0,6.0,1.0,5.0
1,2015.0,11.0,14.0,7.0,1,101.0,5.0,1.0,5.0,2.0,...,29.0,28.0,34.0,26.0,10.0,18.0,13.0,9.0,4.0,6.0
2,2015.0,11.0,15.0,1.0,1,101.0,1.0,1.0,1.0,1.0,...,27.0,22.0,28.0,20.0,19.0,18.0,9.0,7.0,6.0,6.0
3,2015.0,11.0,15.0,1.0,1,101.0,5.0,1.0,4.0,5.0,...,36.0,36.0,51.0,40.0,23.0,15.0,14.0,10.0,6.0,6.0
4,2015.0,11.0,16.0,2.0,1,101.0,1.0,1.0,4.0,6.0,...,27.0,17.0,15.0,11.0,14.0,13.0,9.0,10.0,15.0,13.0


In [94]:
datas.f_system.value_counts()

11          88453
2           86273
1           59278
14          54594
11          43508
12          42084
14          37130
6           30762
2           29361
1           21956
7           21206
16          17326
12          12130
6           11283
16           7994
17           5417
7            5097
8            2656
17           1265
19            874
8             641
9             628
19             60
f_system       49
Name: f_system, dtype: int64

In [97]:
datas.f_system = pd.to_numeric(datas.f_system, errors='coerce', downcast='integer')

In [100]:
datas.describe()

,year_record,month_record,day_record,day_of_week,f_system,station_id,travel_dir,travel_lane,hour_00,hour_01,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
count,579976.0,579976.0,579976.000000,579976.000000,579976.000000,489312.000000,579976.000000,579976.000000,579976.000000,579976.000000,...,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000,579976.000000
mean,2015.0,11.0,15.520947,3.851887,8.110087,147206.060597,3.874086,1.278782,106.396075,74.176118,...,687.727754,733.907019,755.984841,729.174674,587.615724,442.868650,354.111929,294.077501,226.937753,159.304680
std,0.0,0.0,8.588896,2.033455,5.246334,257145.034027,2.283611,1.091038,206.289017,152.531389,...,1006.831266,1047.581690,1067.504026,1046.181255,894.326193,756.853352,632.525413,549.871086,446.827194,322.047491
min,2015.0,11.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,2015.0,11.0,8.000000,2.000000,2.000000,272.000000,1.000000,1.000000,11.000000,7.000000,...,161.000000,172.000000,175.000000,161.000000,117.000000,80.000000,59.000000,44.000000,30.000000,18.000000
50%,2015.0,11.0,15.000000,4.000000,11.000000,6962.000000,4.000000,1.000000,38.000000,25.000000,...,399.000000,427.000000,438.000000,414.000000,315.000000,225.000000,172.000000,134.000000,95.000000,62.000000
75%,2015.0,11.0,23.000000,6.000000,12.000000,140190.000000,5.000000,2.000000,118.000000,81.000000,...,816.000000,883.000000,920.000000,884.000000,694.000000,514.000000,410.000000,336.000000,255.000000,178.000000
max,2015.0,11.0,30.000000,7.000000,19.000000,980503.000000,9.000000,8.000000,19576.000000,18592.000000,...,75401.000000,53001.000000,67101.000000,62101.000000,31901.000000,86500.000000,82100.000000,68800.000000,66900.000000,49104.000000
